In [ ]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
# 연도별로 엑셀자료에 정렬해놓은 데이터 가져오기 (2010 ~ 2019년)
# 회사코드 Symbol에서 앞에 'A'제거하고 숫자만 가져오는 과정

data_2010 = pd.read_excel("코스피200 편입여부.xlsx", sheet_name="2010-01-31")   # 2011-01-31, 2012-01-31, ...
i = 0
for d in data_2010['Symbol']:                           # output은 오류가 뜨지만 'Symbol'앞에 'A'제거하는 과정이 적용된 상태
    data_2010.iloc[i, 0] = data_2010.iloc[i, 0][1:]
    i = i + 1

In [ ]:
# company_code & company_name

K_2010 = data_2010.iloc[:200,0:2]
K_2010_code = list(K_2010["Symbol"])
K_2010_name = list(K_2010["company_name"])
K_2010_new = K_2010.set_index("Symbol")     # 'company_code'를 index로 만들어 제외된 회사, 추가된 회사를 list로 만들기 위한 변수

In [ ]:
# Dart의 IP차단을 막기 위해 갯수 설정

K_2010_code_part = K_2010_code[:40]      # 40 ~ 80 , ... , 160 ~ 200
K_2010_code_part

In [ ]:
# 주소(url) setting

API_KEY = "9a9929b43fd4c79e06cc04d50eb34a70a0a576b1"
start_date = "20100101"
end_date = "20191231"
page_set = "100"   # 페이지당 건수
# auth : 발급받은 인증키(40자리)
# crp_cd : 공시대상회사의 종목코드(상장사:6자리 숫자) 또는 고유번호(기타법인:8자리 숫자)
# start_dt, end_dt : 검색시작,끝 접수일자
# dsn_tp : A001(사업보고서)

url_list = []
for crp in K_2010_code_part:
    company_code = crp   
    url = "http://dart.fss.or.kr/api/search.xml?auth=" + API_KEY + "&crp_cd=" + company_code + "&start_dt=" + start_date + "&end_dt=" + end_date + "&bsn_tp=A001&page_set=" + page_set  
    url_list.append(url)
url_list

In [ ]:
# 각 연도별로 코스피200종목 회사들의 2010 ~ 2019년 모든 사업보고서 리스트를 데이터프레임으로 반환

def data_extract():
    url_lists = url_list
    data_finals = pd.DataFrame()
    for url in url_lists:
        xmlsoup = BeautifulSoup(urlopen(url).read(), 'html.parser')
        te = xmlsoup.findAll("list")
        data = pd.DataFrame()
        for t in te:
            temp = pd.DataFrame(([[t.crp_cls.string, t.crp_nm.string, t.crp_cd.string, t.rpt_nm.string,
                              t.rcp_no.string, t.flr_nm.string, t.rcp_dt.string, t.rmk.string]]),
                              columns = ["crp_cls", "crp_nm", "crp_cd", "rpt_nm", "rcp_no", "flr_nm", "rcp_dt", "rmk"])
            data = pd.concat([data,temp])
            data_result = data
        data_result = data_result[data_result['rmk'] != '정연']   # '정연'은 수정된 사업보고서가 제출되기 이전의 보고서이므로 제외
        data_finals = pd.concat([data_finals, data_result])
        data_finals = data_finals.reset_index(drop=True)
        # 중복된 사업보고서 삭제
        data_finals = data_finals.drop_duplicates(['crp_cls', 'crp_nm', 'crp_cd', 'rpt_nm', 'rcp_no', 'flr_nm', 'rcp_dt', 'rmk'], keep='first')
    return data_finals

In [ ]:
report_result = data_extract()
report_result

In [ ]:
# csv file로 저장

report_result.to_csv('C:/Users/PC/Desktop/2010/K_2010.csv', encoding='euc-kr') # 한글깨짐 방지

#### 위의 과정 2010~2019년도(10번) 동일하게 진행

In [ ]:
# filtering한 데이터 가져오기 (2010년 40개 기업마다 5개, 2011년 40개 기업마다 5개, ..., 2019년 40개 기업마다 5개로 총 50개의 csv파일에 대해)
# (ex) K_2010_2010, K_2010_2010_2, K_2010_2010_3, ... , K_2019_2019_4, K_2019_2019_5

data = pd.read_excel("C:/Users/PC/Desktop/2010/K_2010_2010.xlsx", sheet_name = 'Sheet1', encoding='euc-kr')
data = data.iloc[:-1,:]
data

In [ ]:
# rcp_no로 해당 사업보고서 url과 name 저장

url_code = []
url_name = []
length = len(data)

for t in range(length):
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + str(data['rcp_no'][t])[:-2]
    url_code.append(url)
    name = data['crp_nm'][t]
    url_name.append(name)

In [ ]:
# Dart 의 IP차단을 막기 위해 갯수 설정
part_code = url_code[:20]
part_name = url_name[:20]

In [ ]:
# 사업보고서 url에 그대로 들어가면 각 파트별 HTML코드가 없음.
# 따라서, 각 사업보고서의 HTML 코드에서 '이사의 경영진단 및 분석의견' url을 구한 뒤
# BeautifulSoup으로 HTML코드 partitioning

page_list = []
for code in part_code:
    page = BeautifulSoup(urlopen(code).read(), 'html.parser')  
    body = str(page.find('head')).split('이사의 경영진단 및 분석의견",')[1].split('cnt++')[0].split('viewDoc(')[1].split(')')[0].split(', ')
    body = [body[i][1:-1] for i in range(len(body))]   
    url_final = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=' + body[0] + '&dcmNo=' + body[1] + '&eleId=' + body[2] + '&offset=' + body[3] + '&length=' + body[4] + '&dtd=' + body[5]
    page = BeautifulSoup(urlopen(url_final).read(), 'html.parser')
    page_list.append(page)
    print(url_final)
page_list

In [ ]:
import re
def remove_tag(content):             ## 태그를 제외하고 text만 가져오는 함수
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '\n', content)
    return cleantext
def remove_space(content):          # 개행문자인 \n과 \xa0제거
    cleans = re.compile('[\n]|[\xa0]')
    cleantext = re.sub(cleans, '', content)
    return cleantext

In [ ]:
# partitioning한 코드에서 재무제표 등의 표를 제거하고 텍스트만 가져오기 위해 'table'태그를 제거하는 과정

page_partition = []
for page_char in page_list:
    whole_data = str(page_char).split('<')
    table_data = page_char.find_all('table')
    table_data_li = str(table_data).split('<')
    whole_data_copy = whole_data.copy()

    for table in table_data_li:
        if (table in whole_data) == True:
            whole_data_copy.remove(table)
        
    output = '<'.join(whole_data_copy)

    output1 = remove_tag(output)
    final = remove_space(output1)
    page_partition.append(final)

In [ ]:
page_partition

In [ ]:
# txt file로 저장

result_texts = zip(part_name, page_partition)
for name,text in result_texts:
    file = open('C:/Users/PC/Desktop/2010_text/%s 사업보고서.txt' %name, 'w', encoding='utf-8')
    file.write(text)
    file.close()

#### 2010 ~ 2018년도 KOSPI200에 대해 약 1800개(9 * 200) txt파일 저장
#### 2019년도 사업보고서는 아직 나오지 않음